In [1]:
import json
import pandas as pd
from collections import defaultdict
from nltk.tokenize import word_tokenize
import pickle
import h5py
import re
from utils import img_data_2_mini_batch, imgs2batch
import numpy as np
import matplotlib.pyplot as plt
import torch.utils.data as Data
from torchvision import transforms
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image

%matplotlib inline
import torch.nn.functional as F
import sys, os
from IPython.display import display, HTML
%reload_ext autoreload
%autoreload 2

In [2]:
from data_loader import VQADataSet
pp = lambda parsed: print(json.dumps(parsed, indent=4, sort_keys=True))

In [3]:
dataset = VQADataSet(Q=10)

building QA Maps



build answer vocab



build question vocab)



encode answers



encode questions



VQADataSet init time: 7.025830984115601


In [4]:
# # pp(dataset.answer_maps)
# from random import shuffle
# from collections import defaultdict
# cntr = defaultdict(int)
# dist = defaultdict(list)
# for i, ann in enumerate(dataset.answer_maps):
#     ans = ann['answer']
#     cntr[ans]+=1
#     dist[ans].append(i)

# split = {'train': [], 'test': []}
# z_cnt = 0
# for ans, idxes in dist.items():
#     shuffle(idxes)
#     c = int(len(idxes)*0.20)
#     split['train'] += idxes[c:]
#     split['test'] += idxes[:c]
# split['test'].sort()
# print(split['test'][:20])

# # print()
# # print(z_cnt)
# # print()
# # train = split['train']
# # test = split['test']
# # print(len(split['train']))
# # print(len(split['test']))
# # print(len(split['test'])/(len(train) + len(test)))
# # print(len(train) + len(test))

# def split_to_freq(split, T=None):
#     anns = [dataset.answer_maps[i] for i in split]
#     _cntr = defaultdict(int)
#     for ann in anns:
#         _cntr[ann['answer']]+=1
#     tmp = sorted(_cntr.items(), key=lambda kv: kv[1], reverse=True)[:T]
#     return [x[1] for x in tmp]

# def plot_freq(title, Y):
#     X = range(len(Y))
#     plt.bar(X, Y)
#     plt.title(title)
#     plt.show()

# train_freq = split_to_freq(train, T=50)
# test_freq = split_to_freq(test, T=50)
# plot_freq("answer frequency", train_freq)
# plot_freq("answer frequency", test_freq)
# # T = 100
# # print(len(cntr))
# # kj = sorted(cntr.items(), key=lambda kv: kv[1], reverse=True)
# # # print(kj[:T])
# # import functools as ft
# # Y = list(map(lambda aa: aa[1], kj[:T]))
# # plot_freq(Y)
# # print(ft.reduce(lambda a,b: a+b, kkk))

In [5]:
# from IPython.display import display, HTML

# CSS = """
# .output {
#     flex-direction: row;
# }
# """

# HTML('<style>{}</style>'.format(CSS))

# tmp_dict = defaultdict(int)
# for ann in dataset.anns:
#     tmp_dict[ann['question_type']]+=1

# tmp_res = sorted(tmp_dict.items(), key=lambda kv: kv[1], reverse=True)
# display(pd.DataFrame(tmp_res[:10]))
# display(pd.DataFrame(tmp_res[10:20]))

In [6]:
# dataset._build_vocab()
# print(dataset.vocab['answer'])
# print()
# print(dataset.vocab['question'])

In [7]:
# dataset._encode_qa_and_set_img_path()

In [ ]:
#https://arxiv.org/pdf/1803.07724.pdf